In [1]:
import os
import sys

path = ['..', 'CToxPred']
src_dir = os.path.abspath(os.path.join(*path))
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
import os
print(os.getcwd())

C:\Users\ADS_Lab\Desktop\JH\Paper_Replication\CToxPred\notebooks


In [4]:
import joblib
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torch

from CToxPred.utils import compute_fingerprint_features, compute_descriptor_features, compute_metrics
from CToxPred.hERG_model import hERGClassifier
from CToxPred.nav15_model import Nav15Classifier
from CToxPred.cav12_model import Cav12Classifier

In [5]:
## hERG
eval_herg_set_60 = pd.read_csv("../data/raw/hERG/eval_set_herg_60.csv")
eval_herg_set_70 = pd.read_csv("../data/raw/hERG/eval_set_herg_70.csv")
## Nav1.5
eval_nav_set_60 = pd.read_csv("../data/raw/Nav1.5/eval_set_nav_60.csv")
eval_nav_set_70 = pd.read_csv("../data/raw/Nav1.5/eval_set_nav_70.csv")
## Cav1.2
eval_cav_set_60 = pd.read_csv("../data/raw/Cav1.2/eval_set_cav_60.csv")
eval_cav_set_70 = pd.read_csv("../data/raw/Cav1.2/eval_set_cav_70.csv")

In [6]:
eval_cav_set_70

,InChl Key,SMILES,Source,ChEMBL ID,PubChem CID,pIC50
0,UYZIPLMTFSGKAJ-OAHLLOKOSA-N,N[C@H](COCc1cc(C(F)(F)F)cc(C(F)(F)F)c1)c1ccccc1,ChEMBL,CHEMBL92818,NaN,6.23657
1,YVLZVIUUVUBKNC-QGZVFWFLSA-N,CN(CC(N)=O)[C@H](COCc1cc(C(F)(F)F)cc(C(F)(F)F)...,ChEMBL,CHEMBL327765,NaN,5.00000
2,FLNYLINBEZROPL-NSOVKSMOSA-N,COc1ccccc1CN[C@H]1C2CCN(CC2)[C@H]1C(c1ccccc1)c...,ChEMBL,CHEMBL16192,NaN,6.61979
3,KTQASNDGWLUTPN-UHFFFAOYSA-N,Cc1ccc(CN2CCC(n3c(O)nc4ccccc43)CC2)c(Cl)c1,ChEMBL,CHEMBL114484,NaN,5.87290
4,PAJSQNJPMJTKTN-UHFFFAOYSA-N,Cc1ccc2nc(O)n(C3CCN(Cc4cc5c(cc4Cl)OCO5)CC3)c2c1,ChEMBL,CHEMBL420863,NaN,5.76700
...,...,...,...,...,...,...
76,FTUDAMYKRYOOIG-UHFFFAOYSA-N,COC(=O)C1=CNC=C(C(=O)OC)C1c1ccc(Cl)c(Cl)c1,ChEMBL,CHEMBL602780,NaN,6.00000
77,ZUSRJOLABHKRGS-UHFFFAOYSA-N,CCOC(=O)C1=C(C)NC(C)=C(C(=O)OCC)C1c1ccc(OC)cc1OC,ChEMBL,CHEMBL2063635,NaN,6.00000
78,ODICPULKHOZXSO-UHFFFAOYSA-N,COC(=O)C1=C(C)N(C)C(C)=C(C(=O)OC)C1c1cccc(Cl)c1,ChEMBL,CHEMBL2063807,NaN,6.00000
79,AARJIBDJVXSJTN-UHFFFAOYSA-N,CCOC(=O)C1=C(C)NC(C)=C(C(=O)OC)C1c1ccc(C)o1,ChEMBL,CHEMBL2063639,NaN,6.00000


In [7]:
smiles_list = eval_herg_set_70['SMILES']

In [8]:
fingerprints = compute_fingerprint_features(smiles_list)
descriptors = compute_descriptor_features(smiles_list)

 51%|█████     | 242/473 [00:05<00:02, 93.36it/s]

C:\Users\ADS_Lab\anaconda3\envs\ctoxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ADS_Lab\anaconda3\envs\ctoxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 63%|██████▎   | 298/473 [00:05<00:02, 86.50it/s]

C:\Users\ADS_Lab\anaconda3\envs\ctoxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ADS_Lab\anaconda3\envs\ctoxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 473/473 [00:08<00:00, 54.98it/s]


In [9]:
cav12_fingerprints = fingerprints
cav12_descriptors = descriptors

In [11]:
path = ['..', 'CToxPred', 'models', 'decriptors_preprocessing', 'Cav1.2', 'cav_descriptors_preprocessing_pipeline.sav']

descriptors_transformation_pipeline = joblib.load(os.path.join(*path))
cav12_descriptors = descriptors_transformation_pipeline.transform(cav12_descriptors)

In [14]:
import pandas as pd

# CSV 파일 경로
file_path = 'C:\\Users\\ADS_Lab\\Desktop\\JH\\Paper_Replication\\CToxPred\\dataset\\SkinReaction_train_no_salt.txt'

# 파일 로드 시 첫 번째 열을 'SMILES'로 지정
df = pd.read_csv(file_path, sep='\t', header=None, names=['SMILES', 'Label'])

# 첫 번째 열 확인
print(df.head())


                                              SMILES  Label
0               CCCOc1ccc(Br)c(C(=O)c2ccc(OC)cc2O)c1      0
1                                         CC=C(C)C=O      0
2  O=C1C([PH](c2ccccc2)(c2ccccc2)c2ccccc2)CCN1c1c...      0
3                                   C=C(C=CCC(C)C)CC      0
4                        CCOCc1cc(OC)c(B(O)O)c(OC)c1      0


In [15]:
smiles_list = df['SMILES']

In [16]:
fingerprints = compute_fingerprint_features(smiles_list)
descriptors = compute_descriptor_features(smiles_list)

100%|██████████| 283/283 [00:04<00:00, 70.05it/s] 


In [17]:
SkinReaction_fingerprints = fingerprints
SkinReaction_descriptors = descriptors

In [18]:
path = ['..', 'CToxPred', 'models', 'decriptors_preprocessing', 'Cav1.2', 'cav_descriptors_preprocessing_pipeline.sav']

descriptors_transformation_pipeline = joblib.load(os.path.join(*path))
SkinReaction_descriptors = descriptors_transformation_pipeline.transform(SkinReaction_descriptors)

In [20]:
# 변환된 데이터를 새 .sav 파일로 저장
save_path = os.path.join('C:\\Users\\ADS_Lab\\Desktop\\JH\\Paper_Replication\\CToxPred\\notebooks', 'SkinReaction_descriptors.sav')  # 파일 저장 경로 지정
joblib.dump(SkinReaction_descriptors, save_path)

# 파일이 잘 저장되었는지 확인
print(f"Transformed data saved to: {save_path}")

Transformed data saved to: C:\Users\ADS_Lab\Desktop\JH\Paper_Replication\CToxPred\notebooks\SkinReaction_descriptors.sav


In [22]:
from CToxPred.hERG_model import hERGClassifier
from CToxPred.nav15_model import Nav15Classifier
from CToxPred.cav12_model import Cav12Classifier

import joblib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [23]:
# 데이터셋 로드 및 전처리
def load_data(file_path):
    
    df = pd.read_csv(file_path, sep='\t', header=None, names=['SMILES', 'Label'])
    smiles_list = df['SMILES']
    
    # 예시로 데이터 로드 부분은 새로운 데이터 파일을 읽는 부분입니다.
    # 아래는 파일 경로에서 데이터를 읽고, 전처리하여 반환하는 함수입니다.
    data = pd.read_csv(file_path)
    
    # 필요한 전처리 및 feature 추출 (e.g., fingerprints, descriptors 등)
    fingerprints = compute_fingerprint_features(smiles_list)
    descriptors = compute_descriptor_features(smiles_list)
    
    return fingerprints, descriptors


# 파일 경로 정의
train_file_path = r"C:\Users\ADS_Lab\Desktop\JH\Paper_Replication\CToxPred\dataset\hERG_train_no_salt.txt"
test_file_path = r"C:\Users\ADS_Lab\Desktop\JH\Paper_Replication\CToxPred\dataset\hERG_test_no_salt.txt"

# 학습 데이터 로드
train_dataset = load_data(train_file_path)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 테스트 데이터 로드
test_dataset = load_data(test_file_path)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 모델 초기화
inputSize = train_dataset[0][0].shape[0]  # Morgan Fingerprint 크기 (2048)
outputSize = len(set([y.item() for _, y in train_dataset]))  # 클래스 개수
model = hERGClassifier(inputSize=inputSize, outputSize=outputSize)

# 손실 함수 및 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 저장 및 성능 추적
best_auc = 0.0  # AUC 초기화
best_epoch = 0  # 가장 성능 좋은 모델의 에폭
best_model_path = "best_hERG_classifier_trained.model"  # 가장 성능 좋은 모델 파일 경로

# AUC 계산 함수
def calculate_auc(model, loader):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in loader:
            outputs = model(x_batch)
            preds = outputs[:, 1]  # 양성 클래스 확률
            all_preds.extend(preds.numpy())
            all_labels.extend(y_batch.numpy())

    auc = roc_auc_score(all_labels, all_preds)
    return auc


# 학습
# 학습
epochs = 200  # 에포크 100으로 설정
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 테스트 데이터에 대해 AUC 계산
    auc_value = calculate_auc(model, test_loader)
    print(f"Epoch {epoch + 1}/{epochs}, AUC: {auc_value}")

    # 성능이 가장 좋은 모델 저장
    if auc_value > best_auc:
        best_auc = auc_value
        best_epoch = epoch + 1  # 가장 성능 좋은 모델의 에폭
        torch.save(model.state_dict(), best_model_path)  # AUC 값이 가장 높은 모델 저장

# 가장 성능 좋은 모델의 에폭과 AUC 출력
print(f"Best model achieved at epoch {best_epoch} with AUC: {best_auc}")
print(f"Best model saved at: {best_model_path}")

100%|██████████| 130/130 [00:03<00:00, 35.75it/s]


ValueError: too many values to unpack (expected 2)

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. 데이터 로드 및 전처리
file_path = 'C:\\Users\\ADS_Lab\\Desktop\\JH\\Paper_Replication\\CToxPred\\dataset\\SkinReaction_train_no_salt.txt'

# 파일 로드 시 첫 번째 열을 'SMILES'로 지정
data = pd.read_csv(file_path, sep='\t', header=None, names=['SMILES', 'Label'])

# 데이터 확인
print(data.head())


                                              SMILES  Label
0               CCCOc1ccc(Br)c(C(=O)c2ccc(OC)cc2O)c1      0
1                                         CC=C(C)C=O      0
2  O=C1C([PH](c2ccccc2)(c2ccccc2)c2ccccc2)CCN1c1c...      0
3                                   C=C(C=CCC(C)C)CC      0
4                        CCOCc1cc(OC)c(B(O)O)c(OC)c1      0


In [25]:
# 'target_column_name'을 실제 타겟 열 이름으로 교체
X = data.drop('SMILES', axis=1)  # 'target_column_name'을 타겟 열 이름으로 변경
y = data['Label']  # 타겟 열

In [26]:
# 데이터 나누기 (훈련 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화 (Standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorch Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [27]:
# 3. 모델 초기화 및 학습
input_size = X_train.shape[1]  # 입력 크기 (특징의 개수)
output_size = len(y.unique())  # 출력 크기 (클래스 개수)

model = Cav12Classifier(inputSize=input_size, outputSize=output_size)


In [33]:
from sklearn.metrics import roc_auc_score

# 손실 함수 및 최적화 기법 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # 학습 모드
    optimizer.zero_grad()

    # 순전파
    outputs = model(X_train_tensor)
    
    # 손실 계산
    loss = criterion(outputs, y_train_tensor)
    
    # 역전파
    loss.backward()
    optimizer.step()

    # 예측 확률 계산
    probabilities = torch.softmax(outputs, dim=1).cpu().detach().numpy()

    # AUC 계산
    auc = roc_auc_score(y_train_tensor.cpu().numpy(), probabilities[:, 1])

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, AUC: {auc:.4f}')

Epoch [1/100], Loss: 0.3133, AUC: 1.0000
Epoch [2/100], Loss: 0.3133, AUC: 1.0000
Epoch [3/100], Loss: 0.3133, AUC: 1.0000
Epoch [4/100], Loss: 0.3133, AUC: 1.0000
Epoch [5/100], Loss: 0.3133, AUC: 1.0000
Epoch [6/100], Loss: 0.3133, AUC: 1.0000
Epoch [7/100], Loss: 0.3133, AUC: 1.0000
Epoch [8/100], Loss: 0.3133, AUC: 1.0000
Epoch [9/100], Loss: 0.3133, AUC: 1.0000
Epoch [10/100], Loss: 0.3133, AUC: 1.0000
Epoch [11/100], Loss: 0.3133, AUC: 1.0000
Epoch [12/100], Loss: 0.3133, AUC: 1.0000
Epoch [13/100], Loss: 0.3133, AUC: 1.0000
Epoch [14/100], Loss: 0.3133, AUC: 1.0000
Epoch [15/100], Loss: 0.3133, AUC: 1.0000
Epoch [16/100], Loss: 0.3133, AUC: 1.0000
Epoch [17/100], Loss: 0.3133, AUC: 1.0000
Epoch [18/100], Loss: 0.3133, AUC: 1.0000
Epoch [19/100], Loss: 0.3133, AUC: 1.0000
Epoch [20/100], Loss: 0.3133, AUC: 1.0000
Epoch [21/100], Loss: 0.3133, AUC: 1.0000
Epoch [22/100], Loss: 0.3133, AUC: 1.0000
Epoch [23/100], Loss: 0.3133, AUC: 1.0000
Epoch [24/100], Loss: 0.3133, AUC: 1.0000
E